# Project #3
## Predicting Voting Results

In [26]:
import geopandas as gpd
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import tree
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [40]:
gdf = gpd.read_file('./va_admin_bndry/VirginiaCounty_ClippedToShoreline.shx')
vdf = pd.read_csv('./data/voting_VA.csv')
cdf = pd.read_csv('./data/county_adjacencies.csv')


In [41]:
gdf['FIPS_left'] = pd.to_numeric(gdf['STCOFIPS']) 
df = gdf.merge(cdf,left_on='FIPS_left',right_on='FIPS')

In [42]:
df20 = vdf.loc[ vdf['year']==2020,:]

Dvotes = df20.loc[(df20['party']=='DEMOCRAT'),:].groupby('county_fips')['candidatevotes'].sum()
Dvotes = Dvotes.rename('dem_votes_2020')
df = df.merge(Dvotes,left_on='FIPS_left',right_on='county_fips')

Rvotes = df20.loc[(df20['party']=='REPUBLICAN'),:].groupby('county_fips')['candidatevotes'].sum()
Rvotes = Rvotes.rename('rep_votes_2020')
df = df.merge(Rvotes,left_on='FIPS_left',right_on='county_fips')

In [43]:
df['vote_diff'] = df['rep_votes_2020']-df['dem_votes_2020']

In [44]:
df.head()

,STCOFIPS,GNIS,NAME,NAMELSAD,GSOURCE,LADOPT,LASTUPDATE,JURISTYPE,AREASQMI,Shape_Leng,...,N6,N7,N8,N9,N10,N11,N12,dem_votes_2020,rep_votes_2020,vote_diff
0,51001,1480091,Accomack,Accomack County,T,N,2014-08-20,CO,1195.366291,4.949957e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7578,9172,1594
1,51003,1675170,Albemarle,Albemarle County,L,Y,2021-03-29,CO,725.634334,2.310198e+05,...,Orange,Louisa,Fluvanna,Buckingham,Nelson,NaN,NaN,42466,20804,-21662
2,51005,1492459,Alleghany,Alleghany County,V,N,2014-09-15,CO,449.544769,2.062654e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2243,5859,3616
3,51007,1497770,Amelia,Amelia County,T,Y,2021-03-29,CO,360.945234,1.776617e+05,...,Nottoway,NaN,NaN,NaN,NaN,NaN,NaN,2411,5390,2979
4,51009,1480095,Amherst,Amherst County,V,N,2020-09-11,CO,478.766949,1.876610e+05,...,LynchburgCity,NaN,NaN,NaN,NaN,NaN,NaN,5672,11041,5369


In [47]:
df = df[['rep_votes_2020', 'dem_votes_2020', 'vote_diff']]

TypeError: unhashable type: 'list'

In [32]:
# Split data into training rows and testing rows:
N = df.shape[0]
df = df.sample(frac=1, random_state=100) # randomize the order in which data appears
train_size = int(.8*N)
df_train = df[0:train_size]
N_train = df_train.shape[0]
df_test = df[train_size:]
N_test = df_test.shape[0]

# Bootstrap:
T = 1000
m_depth = 5
Rsq = np.zeros(T) # preallocate the rsq measure
y_test_hat = np.zeros([T,N_test]) # preallocate predictions on test set
df_train.head()

# Split data into train/test:
X_train = df_train.drop(['vote_diff'],axis=1)
y_train = df_train['vote_diff']
X_test = df_test.drop(['vote_diff'],axis=1)
y_test = df_test['vote_diff']



In [33]:
# Bootstrapping:
for s in range(T):
    # Generate a bootstrap sample:
    df_s = df_train.sample(frac=1, replace=True)
    X_s = df_s.drop('vote_diff',axis=1)
    y_s = df_s['vote_diff']
    # Fit decision tree:
    cart = tree.DecisionTreeRegressor(max_depth=m_depth) # Create a classifier object
    cart = cart.fit(X_s, y_s) # Fit the classifier
    # Compute Rsq:
    y_hat = cart.predict(X_test)
    SSE = np.sum( (y_test-y_hat)**2 )
    TSS = np.sum( (y_test-y_s.mean())**2 )
    Rsq[s] = 1 - SSE/TSS
    # Make and Save Predictions:
    y_test_hat[s,:] = y_hat

ValueError: could not convert string to float: 'Hopewell'

In [30]:


# Ensemble predictor:
y_hat_ensemble = y_test_hat.mean(axis=0) # Average the columns to get the ensemble prediction
SSE = np.sum( (y_test-y_hat_ensemble)**2 )
TSS = np.sum( (y_test-y_train.mean())**2 )
Rsq_ensemble = 1 - SSE/TSS

print(Rsq_ensemble)

ValueError: could not convert string to float: 'Mecklenburg'